In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorboard.plugins.hparams import api as hp

from keras.utils import to_categorical
from keras.losses import CategoricalCrossentropy
from keras.models import Model

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as split


from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

Using TensorFlow backend.


In [2]:
np.random.seed(0)

tf.config.optimizer.set_jit(True)
scaler = StandardScaler()

### Conjunto de datos: SDSS DR17

In [3]:
data = pd.read_csv('./SDSS/star_classification.csv')
cols = ['alpha','delta','u','g','r','i','z','redshift','class']
data = data[cols]
data.head()

,alpha,delta,u,g,r,i,z,redshift,class
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,GALAXY
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136,GALAXY
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195,GALAXY
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346,GALAXY
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123,GALAXY


Se cambian las clases a números enteros

$$ \text{GALAXY}: 0,\quad \text{STAR}: 1,\quad \text{QSO}: 2$$

In [4]:
data["class"]=[0 if i == "GALAXY" else 1 if i == "STAR" else 2 for i in data["class"]]
print(data.head())
data = data.to_numpy()

        alpha      delta         u         g         r         i         z  \
0  135.689107  32.494632  23.87882  22.27530  20.39501  19.16573  18.79371   
1  144.826101  31.274185  24.77759  22.83188  22.58444  21.16812  21.61427   
2  142.188790  35.582444  25.26307  22.66389  20.60976  19.34857  18.94827   
3  338.741038  -0.402828  22.13682  23.77656  21.61162  20.50454  19.25010   
4  345.282593  21.183866  19.43718  17.58028  16.49747  15.97711  15.54461   

   redshift  class  
0  0.634794      0  
1  0.779136      0  
2  0.644195      0  
3  0.932346      0  
4  0.116123      0  


### Implementation

We will use wind power forecast data, which is available at the following link. It consists of normalized (between zero and one) wind power measurements from seven wind farms. To keep things simple, we will use first wind farm data (column named wp1) but I encourage the reader to experiment and extend the code to forecast energy for all seven, wind farms.

Let’s import required packages, load the dataset and define two helper functions. The first method prepare_dataset will segment the data into chunks to create X, Y pair for model training. The X will the wind power values from the past (e.g. 1 to t-1) and Y will be future value at time t. The second method train_evaluate perform three things, 1) decoding GA solution to get window size and number of units. 2) Prepare the dataset using window size found by GA and divide into train and validation set, and 3) train LSTM model, calculate RMSE on validation set and return it as a fitness score of the current GA solution.

In [5]:
SC_LAYERS    = np.array([4,8,16,32])
SC_NUM_UNITS = np.array([4,8,16,32])
SC_LEARNING  = np.array([4,8,16,32])*10**(-4)
# SC_BATCHSIZE = np.array([16,32])

callbacks = [keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', mode='max',
                               min_delta=0,
                               patience=3,
                               restore_best_weights=True)]
    
batch_size = 64
epochs = 10

In [6]:
def prepare_dataset(data):
    X, Y = np.empty((0)), np.empty((0))
    X = data[:,0:8]
    Y = data[:,8]
    Y = to_categorical(Y, num_classes=3)
    return X, Y

In [7]:
# Segment the train_data based on new window_size; split into train and validation (70/30)
X,Y = prepare_dataset(data)
X_train, X_test, Y_train, Y_test = split(X, Y, test_size = 0.3, random_state = 0)
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [8]:
def train_evaluate(ga_individual_solution):   
    t = time.time(); t_total = 0
    
    # Decode GA solution to integer for window_size and num_units
    deep_layers_bits = BitArray(ga_individual_solution[0:2])  # 4
    num_units_bits = BitArray(ga_individual_solution[2:4])  # 4
    learning_rate_bits = BitArray(ga_individual_solution[4:6]) # 4
#     batch_size_bits = BitArray(ga_individual_solution[6:7]) # 2
    
    deep_layers = SC_LAYERS[deep_layers_bits.uint]
    num_units = SC_NUM_UNITS[num_units_bits.uint]
    learning_rate = SC_LEARNING[learning_rate_bits.uint]
#     batch_size = SC_BATCHSIZE[batch_size_bits.uint]
    
    print('\n--- Starting trial:', len(datos) )
    print('Deep layers: ',deep_layers,', Num of Units: ',num_units,', Learning rate: ',learning_rate)
    
    # Train LSTM model and predict on validation set
    model = keras.Sequential()
    model.add(Input(shape=(int(X_train.shape[1]),)))
    model.add(Dense(num_units, input_shape=(int(X_train.shape[1]),)))
    
    for i in range(deep_layers):        
        model.add(Dense(num_units, activation='relu'))
    model.add(Dense(3, activation=tf.nn.softmax))
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=["categorical_accuracy"])
    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=batch_size, shuffle=True, verbose=1)
    
    _, score = model.evaluate(X_test, Y_test)    
    t = time.time()-t
    print("Accuracy:", score, "Tiempo de cómputo:", t)

    datos.append([deep_layers, num_units, learning_rate, score, t])
    
    return score,

A continuación, use la paquetería DEAP para definir las cosas para ejecutar GA. Usaremos una representación binaria para la solución de longitud diez. Se inicializará aleatoriamente utilizando la distribución de Bernoulli. Del mismo modo, se utiliza el crossover ordenado, la mutación aleatoria y la selección de la rueda de la ruleta. Los valores del parámetro GA se inicializan arbitrariamente; Te sugerimos que juegues con diferentes configuraciones.



In [9]:
population_size = 10
num_generations = 10
gene_length = 6

datos = []

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = [1.0])
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, 
n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, 
                        ngen = num_generations, verbose = 1)


--- Starting trial: 0
Deep layers:  32 , Num of Units:  32 , Learning rate:  0.0008
Train on 70000 samples, validate on 30000 samples
Epoch 1/10
70000/70000 [==============================] - 27s 392us/sample - loss: 0.6288 - categorical_accuracy: 0.7485 - val_loss: 0.2669 - val_categorical_accuracy: 0.9189
Epoch 2/10
70000/70000 [==============================] - 20s 284us/sample - loss: 0.2128 - categorical_accuracy: 0.9324 - val_loss: 0.4590 - val_categorical_accuracy: 0.9356
Epoch 3/10
70000/70000 [==============================] - 19s 265us/sample - loss: 0.2729 - categorical_accuracy: 0.9056 - val_loss: 0.9661 - val_categorical_accuracy: 0.9456
Epoch 4/10
70000/70000 [==============================] - 18s 257us/sample - loss: 0.1676 - categorical_accuracy: 0.9484 - val_loss: 0.7473 - val_categorical_accuracy: 0.9569
Epoch 5/10
70000/70000 [==============================] - 18s 251us/sample - loss: 0.1498 - categorical_accuracy: 0.9546 - val_loss: 0.6771 - val_categorical_accurac

70000/70000 [==============================] - 8s 118us/sample - loss: 0.1384 - categorical_accuracy: 0.9581 - val_loss: 0.1479 - val_categorical_accuracy: 0.9572
Epoch 4/10
70000/70000 [==============================] - 8s 109us/sample - loss: 0.1306 - categorical_accuracy: 0.9609 - val_loss: 0.1260 - val_categorical_accuracy: 0.9614
Epoch 5/10
70000/70000 [==============================] - 7s 104us/sample - loss: 0.1270 - categorical_accuracy: 0.9616 - val_loss: 0.1554 - val_categorical_accuracy: 0.9518
Epoch 6/10
70000/70000 [==============================] - 7s 98us/sample - loss: 0.1241 - categorical_accuracy: 0.9622 - val_loss: 0.1258 - val_categorical_accuracy: 0.9648
Epoch 7/10
70000/70000 [==============================] - 7s 99us/sample - loss: 0.1202 - categorical_accuracy: 0.9639 - val_loss: 0.1359 - val_categorical_accuracy: 0.9633
Epoch 8/10
70000/70000 [==============================] - 7s 101us/sample - loss: 0.1184 - categorical_accuracy: 0.9645 - val_loss: 0.1470 - va

70000/70000 [==============================] - 6s 86us/sample - loss: 0.1267 - categorical_accuracy: 0.9620 - val_loss: 0.1991 - val_categorical_accuracy: 0.9640
Epoch 6/10
70000/70000 [==============================] - 6s 86us/sample - loss: 0.1218 - categorical_accuracy: 0.9634 - val_loss: 0.1869 - val_categorical_accuracy: 0.9610
Epoch 7/10
30000/30000 [==============================] - 2s 83us/sample - loss: 0.1983 - categorical_accuracy: 0.9644
Accuracy: 0.9644333 Tiempo de cómputo: 48.45868420600891
gen	nevals
0  	10    

--- Starting trial: 10
Deep layers:  8 , Num of Units:  32 , Learning rate:  0.0032
Train on 70000 samples, validate on 30000 samples
Epoch 1/10
70000/70000 [==============================] - 11s 162us/sample - loss: 0.2123 - categorical_accuracy: 0.9258 - val_loss: 0.2106 - val_categorical_accuracy: 0.9500
Epoch 2/10
70000/70000 [==============================] - 8s 115us/sample - loss: 0.1417 - categorical_accuracy: 0.9552 - val_loss: 0.1227 - val_categorical_

70000/70000 [==============================] - 6s 88us/sample - loss: 0.1210 - categorical_accuracy: 0.9640 - val_loss: 0.4541 - val_categorical_accuracy: 0.9654
Epoch 7/10
70000/70000 [==============================] - 6s 86us/sample - loss: 0.1174 - categorical_accuracy: 0.9652 - val_loss: 0.4384 - val_categorical_accuracy: 0.9634
Epoch 8/10
70000/70000 [==============================] - 6s 81us/sample - loss: 0.1158 - categorical_accuracy: 0.9653 - val_loss: 0.4153 - val_categorical_accuracy: 0.9623
Epoch 9/10
70000/70000 [==============================] - 6s 88us/sample - loss: 0.1136 - categorical_accuracy: 0.9662 - val_loss: 0.4516 - val_categorical_accuracy: 0.9665
Epoch 10/10
30000/30000 [==============================] - 2s 80us/sample - loss: 0.4321 - categorical_accuracy: 0.9664
Accuracy: 0.9664 Tiempo de cómputo: 67.36610341072083

--- Starting trial: 15
Deep layers:  32 , Num of Units:  4 , Learning rate:  0.0008
Train on 70000 samples, validate on 30000 samples
Epoch 1/10

70000/70000 [==============================] - 7s 97us/sample - loss: 0.1203 - categorical_accuracy: 0.9645 - val_loss: 0.2414 - val_categorical_accuracy: 0.9633
Epoch 7/10
70000/70000 [==============================] - 6s 88us/sample - loss: 0.1183 - categorical_accuracy: 0.9649 - val_loss: 0.2089 - val_categorical_accuracy: 0.9668
Epoch 8/10
70000/70000 [==============================] - 7s 106us/sample - loss: 0.1161 - categorical_accuracy: 0.9655 - val_loss: 0.2275 - val_categorical_accuracy: 0.9639
Epoch 9/10
70000/70000 [==============================] - 7s 104us/sample - loss: 0.1153 - categorical_accuracy: 0.9660 - val_loss: 0.1986 - val_categorical_accuracy: 0.9659
Epoch 10/10
30000/30000 [==============================] - 3s 95us/sample - loss: 0.2083 - categorical_accuracy: 0.9670
Accuracy: 0.9670333 Tiempo de cómputo: 77.15371227264404

--- Starting trial: 20
Deep layers:  4 , Num of Units:  16 , Learning rate:  0.0008
Train on 70000 samples, validate on 30000 samples
Epoch

...

70000/70000 [==============================] - 13s 185us/sample - loss: 0.2351 - categorical_accuracy: 0.9147 - val_loss: 0.4477 - val_categorical_accuracy: 0.9565
Epoch 2/10
70000/70000 [==============================] - 8s 118us/sample - loss: 0.1388 - categorical_accuracy: 0.9560 - val_loss: 0.1148 - val_categorical_accuracy: 0.9648
Epoch 3/10
70000/70000 [==============================] - 8s 121us/sample - loss: 0.1286 - categorical_accuracy: 0.9601 - val_loss: 0.1177 - val_categorical_accuracy: 0.9637
Epoch 4/10
70000/70000 [==============================] - 8s 120us/sample - loss: 0.1256 - categorical_accuracy: 0.9604 - val_loss: 0.3777 - val_categorical_accuracy: 0.9468
Epoch 5/10
30000/30000 [==============================] - 4s 129us/sample - loss: 0.1148 - categorical_accuracy: 0.9648
Accuracy: 0.9647667 Tiempo de cómputo: 50.68652129173279

--- Starting trial: 35
Deep layers:  4 , Num of Units:  4 , Learning rate:  0.0004
Train on 70000 samples, validate on 30000 samples
Epo

70000/70000 [==============================] - 8s 112us/sample - loss: 0.1339 - categorical_accuracy: 0.9605 - val_loss: 0.1804 - val_categorical_accuracy: 0.9596
Epoch 7/10
70000/70000 [==============================] - 8s 112us/sample - loss: 0.1280 - categorical_accuracy: 0.9628 - val_loss: 0.1580 - val_categorical_accuracy: 0.9576
Epoch 8/10
70000/70000 [==============================] - 8s 112us/sample - loss: 0.1243 - categorical_accuracy: 0.9637 - val_loss: 0.1619 - val_categorical_accuracy: 0.9654
Epoch 9/10
70000/70000 [==============================] - 8s 112us/sample - loss: 0.1212 - categorical_accuracy: 0.9651 - val_loss: 0.2169 - val_categorical_accuracy: 0.9650
Epoch 10/10
30000/30000 [==============================] - 3s 109us/sample - loss: 0.1830 - categorical_accuracy: 0.9661
Accuracy: 0.96606666 Tiempo de cómputo: 85.42602920532227

--- Starting trial: 39
Deep layers:  4 , Num of Units:  8 , Learning rate:  0.0004
Train on 70000 samples, validate on 30000 samples
Ep

70000/70000 [==============================] - 9s 128us/sample - loss: 0.1273 - categorical_accuracy: 0.9626 - val_loss: 0.1421 - val_categorical_accuracy: 0.9648
Epoch 7/10
70000/70000 [==============================] - 9s 128us/sample - loss: 0.1235 - categorical_accuracy: 0.9632 - val_loss: 0.1507 - val_categorical_accuracy: 0.9581
Epoch 8/10
70000/70000 [==============================] - 9s 128us/sample - loss: 0.1227 - categorical_accuracy: 0.9637 - val_loss: 0.1553 - val_categorical_accuracy: 0.9617
Epoch 9/10
70000/70000 [==============================] - 9s 128us/sample - loss: 0.1203 - categorical_accuracy: 0.9644 - val_loss: 0.1369 - val_categorical_accuracy: 0.9648
Epoch 10/10
30000/30000 [==============================] - 3s 113us/sample - loss: 0.1359 - categorical_accuracy: 0.9639
Accuracy: 0.9639 Tiempo de cómputo: 99.59833788871765

--- Starting trial: 43
Deep layers:  4 , Num of Units:  4 , Learning rate:  0.0004
Train on 70000 samples, validate on 30000 samples
Epoch 

70000/70000 [==============================] - 8s 111us/sample - loss: 0.1286 - categorical_accuracy: 0.9623 - val_loss: 0.3356 - val_categorical_accuracy: 0.9631
Epoch 7/10
70000/70000 [==============================] - 8s 111us/sample - loss: 0.1245 - categorical_accuracy: 0.9635 - val_loss: 0.3847 - val_categorical_accuracy: 0.9652
Epoch 8/10
70000/70000 [==============================] - 8s 111us/sample - loss: 0.1209 - categorical_accuracy: 0.9644 - val_loss: 0.3832 - val_categorical_accuracy: 0.9644
Epoch 9/10
70000/70000 [==============================] - 8s 111us/sample - loss: 0.1186 - categorical_accuracy: 0.9648 - val_loss: 0.3842 - val_categorical_accuracy: 0.9613
Epoch 10/10
30000/30000 [==============================] - 3s 108us/sample - loss: 0.5234 - categorical_accuracy: 0.9675
Accuracy: 0.96753335 Tiempo de cómputo: 83.22535181045532

--- Starting trial: 47
Deep layers:  4 , Num of Units:  8 , Learning rate:  0.0008
Train on 70000 samples, validate on 30000 samples
Ep

70000/70000 [==============================] - 9s 130us/sample - loss: 0.3520 - categorical_accuracy: 0.9295 - val_loss: 0.3324 - val_categorical_accuracy: 0.9302
Epoch 7/10
70000/70000 [==============================] - 9s 129us/sample - loss: 0.3369 - categorical_accuracy: 0.9305 - val_loss: 0.3198 - val_categorical_accuracy: 0.9297
Epoch 8/10
70000/70000 [==============================] - 9s 124us/sample - loss: 0.3218 - categorical_accuracy: 0.9308 - val_loss: 0.3020 - val_categorical_accuracy: 0.9338
Epoch 9/10
70000/70000 [==============================] - 9s 124us/sample - loss: 0.2996 - categorical_accuracy: 0.9371 - val_loss: 0.2658 - val_categorical_accuracy: 0.9434
Epoch 10/10
30000/30000 [==============================] - 4s 121us/sample - loss: 0.2145 - categorical_accuracy: 0.9477
Accuracy: 0.9477 Tiempo de cómputo: 96.68121480941772
10 	5     


### Guardar datos

In [11]:
filename = "historial_sdss.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Accuracy", "Tiempo de ejecución"])

df.sort_values(by=["Accuracy", "Tiempo de ejecución"], ascending=[0,0], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [12]:
df

,Deep size,Num units,Learning rate,Accuracy,Tiempo de ejecución
0,4,32,0.0004,0.969500,91.885597
1,4,16,0.0032,0.968733,66.283666
2,4,16,0.0004,0.967800,66.943443
3,4,8,0.0004,0.967533,83.225352
4,4,16,0.0004,0.967367,66.940458
5,4,16,0.0008,0.967367,66.925971
6,16,32,0.0004,0.967233,104.694810
7,8,8,0.0004,0.967033,77.153712
8,4,8,0.0004,0.966733,84.571752
9,4,16,0.0004,0.966733,67.264888


In [13]:
np.sum(df[["Tiempo de ejecución"]])/60/60

Tiempo de ejecución    1.208307
dtype: float64

### Cargar datos

In [ ]:
pathname = os.path.join('.\\', filename)
while True:
    try: 
        df 
        break
    except:
        df = pd.read_csv(pathname, delimiter = "\t")
        break

In [ ]:
df

### Mejores individuos

In [ ]:
# Selecciona los mejores k individuos - (para k=5)
k = 3
best_genes = df.iloc[:k,]

best_deep_size = best_genes.iloc[:,0]
best_num_units = best_genes.iloc[:,1]
best_learning_rate = best_genes.iloc[:,2]
best_batch_size = best_genes.iloc[:,3]
best_epochs = best_genes.iloc[:,4]

best_genes

In [ ]:
best_deep_size

In [ ]:
# Train the model using best configuration on complete training set 
#and make predictions on the test set

X,Y = prepare_dataset(train_data)
X_train, X_test, y_train, y_test = split(X, Y, test_size = 0.20, random_state = 0)

models = []
historial = []
y_pred = []

for k in range(len(best_deep_size)):
    print('\nDeep Size: ', best_deep_size[k], ', Num of Units: ', best_num_units[k], ', Learning rate: ', best_learning_rate[k])
    print('Batch Size: ', best_batch_size[k], ", Num of Epochs: ", best_epochs[k])
    
    models.append(keras.Sequential())
    models[-1].add(keras.Input(shape=(int(X_train.shape[1]),)))
    models[-1].add(layers.Dense(best_num_units[k], input_shape=(int(X_train.shape[1]),)))
#     x = LSTM(num_units, input_shape=(window_size,1))(inputs)

    for i in range(best_deep_size[k]):        
        models[-1].add(layers.Dense(best_num_units[k], activation='relu'))
    models[-1].add(layers.Dense(1, activation='linear'))

    optimizer = keras.optimizers.Adam(learning_rate=best_learning_rate[k], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    models[-1].compile(optimizer=optimizer, loss=metrics[1][m])
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                       min_delta=0,
                                       patience=50,
                                       restore_best_weights=True)]
#     history = models[-1].fit(X_train, y_train, validation_data=(X_test, y_test), 
#                         epochs=best_epochs, callbacks=callbacks, batch_size=best_batch_size, shuffle=True)
    historial.append(models[-1].fit(X_train, y_train, validation_data=(X_test, y_test),
                        epochs=best_epochs[k], batch_size=best_batch_size[k], shuffle=True))
    y_pred.append(models[-1].predict(X))

    error = metrics[0][m](Y, y_pred[-1])
    print('Test RMSE(RMSLE): ', error)

In [ ]:
fig, axs = plt.subplots(1, len(models), figsize=(14,5), sharey='row')
fig.suptitle('Best models')
titles = []
for k in range(len(models)):
    titles.append(f"Model loss {k+1} (log)")
    axs[k].plot(historial[k].history['loss'])
    axs[k].plot(historial[k].history['val_loss'])
#     print(titles[-1])
    axs[k].set_title(titles[-1])
    axs[k].set_yscale("log")
    axs[k].legend(['train', 'test'], loc='upper left')
    axs[k].grid()

for ax in axs.flat:
    ax.set(xlabel='Epoch', ylabel='Loss')
plt.show()

### Gráficas de comparación

In [ ]:
Y_pred1 = models[0].predict(X)
Y_pred2 = models[1].predict(X)
Y_pred3 = models[2].predict(X)

In [ ]:
fig, axs = plt.subplots(len(models), figsize=(14,9), sharex="col")
fig.suptitle('Best models')

axs[0].scatter(X[:,0], Y, s=1e1, alpha=0.8, c='red', label="Data")
axs[0].set_yscale("log"); axs[0].grid(); axs[0].legend()

axs[1].scatter(X[:,0], Y_pred1, s=1e1, alpha=0.8, c='blue', label="Model pred 1")
axs[1].set_yscale("log"); axs[1].grid(); axs[1].legend()

axs[2].scatter(X[:,0], Y_pred3, s=1e1, alpha=0.8, c='purple', label="Model pred 3")
axs[2].set_yscale("log"); axs[2].grid(); axs[2].legend()

# axs[3].scatter(X[:,0], Y_pred3, s=1e1, alpha=0.8, c='green', label="Model pred 3")
# axs[3].set_yscale("log"); axs[3].grid(); axs[3].legend()

for ax in axs.flat:
    ax.set(xlabel=r'$k$', ylabel='$A(k,r,x)$')
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(X[:,0], Y, s=5e1, alpha=0.8, c='red', label="Data")
# plt.scatter(X[:,0], Y_pred1, s=1e1, alpha=0.4, c='blue', label="Model pred 1")
plt.scatter(X[:,0], Y_pred3, s=1e1, alpha=0.4, c='green', label="Model pred 3")
plt.xlabel('$k$', fontsize=14); plt.ylabel(r'$A(k,r,x)$', fontsize=14)
plt.yscale('log'); plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(len(models), figsize=(14,9), sharex="col")
fig.suptitle('Data and model predictions')

axs[0].scatter(X[:,1], Y, s=1e1, alpha=0.8, c='red', label="Data")
axs[0].set_yscale("log"); axs[0].grid(); axs[0].legend()

axs[1].scatter(X[:,1], Y_pred1, s=1e1, alpha=0.8, c='blue', label="Model pred 1")
axs[1].set_yscale("log"); axs[1].grid(); axs[1].legend()

axs[2].scatter(X[:,1], Y_pred3, s=1e1, alpha=0.8, c='purple', label="Model pred 3")
axs[2].set_yscale("log"); axs[2].grid(); axs[2].legend()

for ax in axs.flat:
    ax.set(xlabel=r'$r$', ylabel='$A(k,r,x)$')
plt.show()

In [ ]:
plt.figure(figsize=(14,7))
plt.scatter(X[:,1], Y, s=5e0, alpha=0.7, c='red', label="Data")
plt.scatter(X[:,1], Y_pred3, s=5e0, alpha=0.7, c='green', label="Model pred 3")
# plt.scatter(X[:,1], Y_pred2, s=5e0, alpha=0.7, c='blue', label="Model pred 2")
plt.xlabel('$r$', fontsize=14); plt.ylabel(r'$A(k,r,x)$', fontsize=14)
plt.yscale('log'); plt.legend(); plt.grid();
plt.show()

In [ ]:
fig, axs = plt.subplots(1,len(models), figsize=(14,5), sharey="row")
fig.suptitle('Data and model predictions')

axs[0].scatter(X[:,2], Y, s=1e0, alpha=0.8, c='red', label="Data")
axs[0].set_yscale("log"); axs[0].grid(); axs[0].legend()

axs[1].scatter(X[:,2], Y_pred1, s=1e0, alpha=0.8, c='blue', label="Model pred 1")
axs[1].set_yscale("log"); axs[1].grid(); axs[1].legend()

axs[2].scatter(X[:,2], Y_pred3, s=1e0, alpha=0.8, c='purple', label="Model pred 3")
axs[2].set_yscale("log"); axs[2].grid(); axs[2].legend()

for ax in axs.flat:
    ax.set(xlabel=r'$x$', ylabel='$A(k,r,x)$')
plt.show()

### Comparación 3D

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X[:,0], X[:,1], X[:,2], marker='+', c=Y, cmap='seismic', alpha=1)
ax.set_xlabel('$k$', fontsize=15)
ax.set_ylabel('$r$', fontsize=15)
ax.set_zlabel('$x$', fontsize=15)
plt.colorbar(p, shrink=0.5, label='$A(k, r, x)$'); plt.title(r"Data")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X[:,0], X[:,1], X[:,2], marker='+', c=Y_pred1, cmap='seismic', alpha=1)
ax.set_xlabel('$k$', fontsize=15)
ax.set_ylabel('$r$', fontsize=15)
ax.set_zlabel('$x$', fontsize=15)
plt.colorbar(p, shrink=0.5, label='$A(k, r, x)$'); plt.title(r"Model prediction 1")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(111, projection='3d')
p = ax.scatter(X[:,0], X[:,1], X[:,2], marker='+', c=Y_pred3, cmap='seismic', alpha=1)
ax.set_xlabel('$k$', fontsize=15)
ax.set_ylabel('$r$', fontsize=15)
ax.set_zlabel('$x$', fontsize=15)
plt.colorbar(p, shrink=0.5, label='$A(k, r, x)$'); plt.title(r"Model prediction 3")
plt.show()

### Resultado de hiperparámetros

In [ ]:
import seaborn as sns

In [ ]:
df

In [ ]:
# Guarda las mejores N soluciones - (1, para k=1)
best_individuals2 = tools.selBest(population,k = 20)
deep_size2 = []
num_units2 = []
learning_rate2 = []
batch_size2 = []
epochs2 = []
t = 0

for bi in best_individuals2:
    deep_size_bits2 = BitArray(bi[0:2])
    num_units_bits2 = BitArray(bi[2:4])
    learning_rate_bits2 = BitArray(bi[4:7])
    batch_size_bits2 = BitArray(bi[7:9])
    epochs_bits2 = BitArray(bi[9:])
    t += 1 
    
    deep_size2.append( 4*(deep_size_bits2.uint + 1))
    num_units2.append( 4*(num_units_bits2.uint + 1))
    learning_rate2.append( 8*(learning_rate_bits2.uint+1)*10**(-4))
    batch_size2.append( 2**(batch_size_bits2.uint +1))
    epochs2.append( 50*(epochs_bits2.uint +1))
#     print('k=',t,'\nDeep Size: ', best_deep_size2[-1], ', Num of Units: ', best_num_units2[-1], ', Learning rate: ', best_learning_rate2[-1])
#     print('Batch Size: ', best_batch_size2[-1], ", Num of Epochs: ", best_epochs2[-1])

In [ ]:

best_genes.iloc[0]

In [ ]:
# fig, axs = plt.subplots(1,2, figsize=(15,5))

# fig.suptitle('Última generación de hipermarámetros obtenidos')

# # axs[0].plot(df.iloc[:,0], df.iloc[:,1], alpha=0.5, c='blue', label="Model pred 1")
# axs[0].scatter(df.iloc[:,0], df.iloc[:,1], s=20, alpha=0.3, c='red', label="Model pred 1")
# axs[0].scatter(best_genes.iloc[:,0], best_genes.iloc[:,1], s=80, alpha=1, c='red', label="Mejores individuos")
# axs[0].grid(); axs[0].legend()
# # axs[0].legend(loc='upper right'); 
# # axs[0].set_xlim([2, 18]); axs[0].set_ylim([1, 65])
# axs[0].set(xlabel=r'Deep size', ylabel='Num units')

# # axs[1].plot(df.iloc[:,2], df.iloc[:,3], alpha=0.5, c='blue', label="Model pred 1")
# axs[1].scatter(df.iloc[:,2], df.iloc[:,3], s=20, alpha=0.3, c='red', label="Model pred 1")
# axs[1].scatter(best_genes.iloc[:,2], best_genes.iloc[:,3], s=80, alpha=1, c='red', label="Mejores individuos")
# axs[1].set_yscale("log"); 
# axs[1].grid(); axs[1].legend(loc='upper right')
# # axs[1].set_xlim([1e-4, 31e-4]); axs[1].set_ylim([2**1, 2**4])
# axs[1].set(xlabel=r'Learning rate', ylabel='Batch size')

In [ ]:
g1= sns.jointplot(data=df, x="Deep size", y="Num units", kind="kde")
g1.ax_joint.scatter(best_genes.iloc[:,0], best_genes.iloc[:,1], color = 'red', label="a")

plt.show()

g2= sns.jointplot(x=df.iloc[:,2], y=df.iloc[:,3], kind='kde')
g2.ax_joint.scatter(best_genes.iloc[:,2], best_genes.iloc[:,3], color = 'red', label="a")

plt.show()